In [18]:
!pip install torch torchvision
!pip install ultralytics

In [19]:
import torch
import shutil
from pathlib import Path
from ultralytics import YOLO

# Set directories
train_dir = Path("Split/train")
val_dir = Path("Split/validation")
test_dir = Path("Split/test")

In [20]:
def organize_yolo_data(data_dir, output_dir):
    output_dir.mkdir(parents=True, exist_ok=True)
    for category in ['normal', 'abnormal']:
        for img_file in (data_dir / category).glob('*.jpg'):
            label = 0 if category == 'normal' else 1
            img_output_dir = output_dir / 'images'
            img_output_dir.mkdir(parents=True, exist_ok=True)
            shutil.copy(img_file, img_output_dir / img_file.name)

            label_output_dir = output_dir / 'labels'
            label_output_dir.mkdir(parents=True, exist_ok=True)
            label_file = label_output_dir / (img_file.stem + '.txt')
            with open(label_file, 'w') as f:
                f.write(f"{label} 0.5 0.5 1 1")

# Prepare data for YOLO
yolo_data_dir = Path("yolo_data")
organize_yolo_data(train_dir, yolo_data_dir / 'train')
organize_yolo_data(val_dir, yolo_data_dir / 'val')
organize_yolo_data(test_dir, yolo_data_dir / 'test')

In [30]:
import os
import shutil

# Define the paths
dataset_path = 'Split'
yolo_data_path = 'datasets/yolo_data'
subfolders = ['train', 'test', 'validation']
categories = {'normal': 0, 'abnormal': 1}

# Create the YOLO directories
for subfolder in subfolders:
    os.makedirs(os.path.join(yolo_data_path, subfolder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(yolo_data_path, subfolder, 'labels'), exist_ok=True)

# Function to process and copy images and create labels
def process_images(subfolder):
    for category in categories:
        category_path = os.path.join(dataset_path, subfolder, category)
        label_value = categories[category]
        
        for image_name in os.listdir(category_path):
            image_path = os.path.join(category_path, image_name)
            
            if os.path.isfile(image_path):
                # Define new paths
                new_image_path = os.path.join(yolo_data_path, subfolder, 'images', image_name)
                label_name = os.path.splitext(image_name)[0] + '.txt'
                label_path = os.path.join(yolo_data_path, subfolder, 'labels', label_name)
                
                # Copy image
                shutil.copyfile(image_path, new_image_path)
                
                # Create label
                with open(label_path, 'w') as label_file:
                    label_file.write(f"{label_value} 0.5 0.5 1 1")

# Process each subfolder
for subfolder in subfolders:
    process_images(subfolder)


In [38]:
import os

# Define the dataset_yaml content
dataset_yaml = """
path: yolo_data
train: train/images
val: validation/images
test: test/images
nc: 2
names: ['normal', 'abnormal']
"""

# Define the directory and file path
dataset_yaml_dir = 'dataset\yolo_data'
dataset_yaml_file = os.path.join(dataset_yaml_dir, 'dataset.yaml')

# Ensure the directory exists
os.makedirs(dataset_yaml_dir, exist_ok=True)

# Write the dataset_yaml content to the file
with open(dataset_yaml_file, 'w') as f:
    f.write(dataset_yaml)

print(f"dataset.yaml has been written to {dataset_yaml_file}")


dataset.yaml has been written to dataset\yolo_data\dataset.yaml


In [33]:
model = YOLO('yolov5su.pt')
model.train(data=str(dataset_yaml_file), epochs=10, batch=16, imgsz=640, name='train4', verbose=True)


New https://pypi.org/project/ultralytics/8.2.48 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.42  Python-3.11.4 torch-2.3.1+cpu CPU (AMD Ryzen 5 4600H with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov5su.pt, data=yolo_data\dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train45, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fal

100%|██████████| 755k/755k [00:00<00:00, 1.44MB/s]



Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      3520  ultralytics.nn.modules.conv.Conv             [3, 32, 6, 2, 2]              
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     18816  ultralytics.nn.modules.block.C3              [64, 64, 1]                   
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    115712  ultralytics.nn.modules.block.C3              [128, 128, 2]                 
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  3    625152  ultralytics.nn.modules.block.C3              [256, 256, 3]                 
  7                  -1  1   1180672  ultralytic

train: Scanning E:\Study\Project\NPL(Gastric cancer)\datasets\yolo_data\train\labels... 704 images, 0 backgrounds, 0 corrupt: 100%|██████████| 704/704 [00:01<00:00, 424.36it/s]


train: New cache created: E:\Study\Project\NPL(Gastric cancer)\datasets\yolo_data\train\labels.cache


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/10         0G     0.0629      0.496      0.919         16        640: 100%|██████████| 44/44 [12:03<00:00, 16.45s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:45<00:00,  9.16s/it]

                   all        152        152      0.811      0.852      0.886      0.877



10 epochs completed in 2.254 hours.
Optimizer stripped from runs\detect\train45\weights\last.pt, 18.5MB
Optimizer stripped from runs\detect\train45\weights\best.pt, 18.5MB

Validating runs\detect\train45\weights\best.pt...
Ultralytics YOLOv8.2.42  Python-3.11.4 torch-2.3.1+cpu CPU (AMD Ryzen 5 4600H with Radeon Graphics)
YOLOv5s summary (fused): 193 layers, 9112310 parameters, 0 gradients, 23.8 GFLOPs


FileNotFoundError: 'yolo_data\dataset.yaml' does not exist

In [34]:
results = model.val()
print(f"Results: {results}")

Ultralytics YOLOv8.2.42  Python-3.11.4 torch-2.3.1+cpu CPU (AMD Ryzen 5 4600H with Radeon Graphics)
YOLOv5s summary (fused): 193 layers, 9112310 parameters, 13254 gradients, 23.8 GFLOPs

Dataset 'coco.yaml' images not found , missing path 'E:\Study\Project\NPL(Gastric cancer)\datasets\coco\val2017.txt'


100%|██████████| 169M/169M [01:57<00:00, 1.50MB/s] 
Unzipping E:\Study\Project\NPL(Gastric cancer)\datasets\coco2017labels-segments.zip to E:\Study\Project\NPL(Gastric cancer)\datasets\coco...: 100%|██████████| 122232/122232 [01:21<00:00, 1491.29file/s]


Dataset download success  (9490.2s), saved to E:\Study\Project\NPL(Gastric cancer)\datasets



val: Scanning E:\Study\Project\NPL(Gastric cancer)\datasets\coco\labels\val2017... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:13<00:00, 364.84it/s]


val: New cache created: E:\Study\Project\NPL(Gastric cancer)\datasets\coco\labels\val2017.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/313 [00:01<?, ?it/s]


IndexError: index 22 is out of bounds for axis 1 with size 3

In [ ]:
model = YOLO("runs/detect/train3/weights/best.pt")
test_results = model('/content/yolo_data/test/images')

In [ ]:
# Run inference on the test set and collect predictions
predictions = []
true_labels = []

for img_file in (test_dir / 'abnormal').glob('*.jpg'):
    result = model(img_file)
    pred = 1 if len(result[0].boxes) > 0 else 0
    predictions.append(pred)
    true_labels.append(1)

for img_file in (test_dir / 'normal').glob('*.jpg'):
    result = model(img_file)
    pred = 1 if len(result[0].boxes) > 0 else 0
    predictions.append(pred)
    true_labels.append(0)


In [ ]:
from sklearn.metrics import classification_report

report = classification_report(true_labels, predictions)
print("Classification Report:\n", report)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Calculate confusion matrix
cm = confusion_matrix(true_labels, pred_labels, labels=[0, 1])

# Plot confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d', xticklabels=['Normal', 'Abnormal'], yticklabels=['Normal', 'Abnormal'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Plot the graphs
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(results_df['         train/box_loss'], label='Training Loss')
plt.plot(results_df['           val/box_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(results_df['       metrics/mAP50(B)'], label='Accuracy (mAP50)')
plt.plot(results_df['    metrics/mAP50-95(B)'], label='Accuracy (mAP50-95)')
plt.title('Accuracy Metrics')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
import pandas as pd

# Load training results
results_df = pd.read_csv('/content/runs/detect/train/results.csv')

# Print final metrics
final_epoch = results_df.iloc[-1]
print(f"Final Training Loss: {final_epoch['         train/box_loss']:.4f}")
print(f"Final Validation Loss: {final_epoch['           val/box_loss']:.4f}")
print(f"Final Accuracy (mAP50): {final_epoch['       metrics/mAP50(B)']:.4f}")
print(f"Final Accuracy (mAP50-95): {final_epoch['    metrics/mAP50-95(B)']:.4f}")